##### ----- 從josn檔案中讀取'text'、'tweet_id'
大致步驟

1. 從json中讀取text和tweet_id並儲存成list

2. 'text'和'tweet_id'合併成pandas dataframe，得到'Total_df'

In [ ]:
import json

data_list = []

with open("tweets_DM.json", 'r') as file:
    for line in file:
        data = json.loads(line)
        data_list.append(data)

In [119]:
# data_list 包含了 JSON 檔案中的所有對象
print(len(data_list))
dic = data_list[0]
print(type(dic))
dic

1867535
<class 'dict'>


{'_score': 391,
 '_index': 'hashtag_tweets',
 '_source': {'tweet': {'hashtags': ['Snapchat'],
   'tweet_id': '0x376b20',
   'text': 'People who post "add me on #Snapchat" must be dehydrated. Cuz man.... that\'s <LH>'}},
 '_crawldate': '2015-05-23 11:42:47',
 '_type': 'tweets'}

In [121]:
data_list[0]['_source']['tweet']['text']

'People who post "add me on #Snapchat" must be dehydrated. Cuz man.... that\'s <LH>'

In [ ]:
import pandas as pd

tweet_id_tmp = []
text_tmp = []

for data in data_list:
    tweet_id_tmp.append(data['_source']['tweet']['tweet_id'])
    text_tmp.append(data['_source']['tweet']['text'])

Total_df = pd.DataFrame({
    'tweet_id':tweet_id_tmp,
    'text':text_tmp,
})

Total_df

,tweet_id,text
0,0x376b20,"People who post ""add me on #Snapchat"" must be ..."
1,0x2d5350,"@brianklaas As we see, Trump is dangerous to #..."
2,0x28b412,"Confident of your obedience, I write to you, k..."
3,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>
4,0x2de201,"""Trust is not the same as faith. A friend is s..."
...,...,...
1867530,0x316b80,When you buy the last 2 tickets remaining for ...
1867531,0x29d0cb,I swear all this hard work gone pay off one da...
1867532,0x2a6a4f,@Parcel2Go no card left when I wasn't in so I ...
1867533,0x24faed,"Ah, corporate life, where you can date <LH> us..."


##### ----- 讀取'emotion.csv'和'data_identification.csv'檔案，並與Total_df合併

大致步驟

1. 以pd.read_csv()讀取兩個csv檔案

2. 用pd.merge()以'tweet_id'合併'Total_df'和'identification_csv'，得到'merged_df'

3. 並以DataFrame.loc[DataFrame["identification"]=='train']來分割train與test的data，得到'train_df'、'test_df'

4. 最後再用pd.merge()以'tweet_id'合併'train_df'和'emotion_csv'，得到'train_df'

5. 刪除'identification'欄位，並儲存成'.pkl'檔案，方便後續取用

In [ ]:
emotion_csv = pd.read_csv("emotion.csv", header=None, names=["tweet_id", "emotion"], skiprows=[0])
identification_csv = pd.read_csv("data_identification.csv", header=None, names=["tweet_id", "identification"], skiprows=[0])

print(pd.DataFrame(emotion_csv))
print("\n")
print(pd.DataFrame(identification_csv))


         tweet_id       emotion
0        0x3140b1       sadness
1        0x368b73       disgust
2        0x296183  anticipation
3        0x2bd6e1           joy
4        0x2ee1dd  anticipation
...           ...           ...
1455558  0x38dba0           joy
1455559  0x300ea2           joy
1455560  0x360b99          fear
1455561  0x22eecf           joy
1455562  0x2fb282  anticipation

[1455563 rows x 2 columns]


         tweet_id identification
0        0x28cc61           test
1        0x29e452          train
2        0x2b3819          train
3        0x2db41f           test
4        0x2a2acc          train
...           ...            ...
1867530  0x227e25          train
1867531  0x293813          train
1867532  0x1e1a7e          train
1867533  0x2156a5          train
1867534  0x2bb9d2          train

[1867535 rows x 2 columns]


In [ ]:
identification_csv.loc[identification_csv['tweet_id']=='0x376b20']

,tweet_id,identification
193653,0x376b20,train


In [ ]:
# 使用 merge 合併兩個 DataFrame，根據 'id' 列進行合併
merged_df = pd.merge(Total_df, identification_csv, on='tweet_id')
merged_df

,tweet_id,text,identification
0,0x376b20,"People who post ""add me on #Snapchat"" must be ...",train
1,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",train
2,0x28b412,"Confident of your obedience, I write to you, k...",test
3,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,train
4,0x2de201,"""Trust is not the same as faith. A friend is s...",test
...,...,...,...
1867530,0x316b80,When you buy the last 2 tickets remaining for ...,test
1867531,0x29d0cb,I swear all this hard work gone pay off one da...,test
1867532,0x2a6a4f,@Parcel2Go no card left when I wasn't in so I ...,test
1867533,0x24faed,"Ah, corporate life, where you can date <LH> us...",train


In [ ]:
train_df =merged_df.loc[merged_df["identification"]=='train']
test_df = merged_df.loc[merged_df["identification"]=='test']

train_df

,tweet_id,text,identification
0,0x376b20,"People who post ""add me on #Snapchat"" must be ...",train
1,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",train
3,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,train
5,0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,train
6,0x2c91a8,Still waiting on those supplies Liscus. <LH>,train
...,...,...,...
1867526,0x321566,I'm SO HAPPY!!! #NoWonder the name of this sho...,train
1867527,0x38959e,In every circumtance I'd like to be thankful t...,train
1867528,0x2cbca6,there's currently two girls walking around the...,train
1867533,0x24faed,"Ah, corporate life, where you can date <LH> us...",train


In [ ]:
test_df

,tweet_id,text,identification
2,0x28b412,"Confident of your obedience, I write to you, k...",test
4,0x2de201,"""Trust is not the same as faith. A friend is s...",test
9,0x218443,When do you have enough ? When are you satisfi...,test
30,0x2939d5,"God woke you up, now chase the day #GodsPlan #...",test
33,0x26289a,"In these tough times, who do YOU turn to as yo...",test
...,...,...,...
1867525,0x2913b4,"""For this is the message that ye heard from th...",test
1867529,0x2a980e,"""There is a lad here, which hath five barley l...",test
1867530,0x316b80,When you buy the last 2 tickets remaining for ...,test
1867531,0x29d0cb,I swear all this hard work gone pay off one da...,test


In [ ]:
train_df = pd.merge(train_df, emotion_csv, on='tweet_id')

train_df

,tweet_id,text,identification,emotion
0,0x376b20,"People who post ""add me on #Snapchat"" must be ...",train,anticipation
1,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",train,sadness
2,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,train,fear
3,0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,train,joy
4,0x2c91a8,Still waiting on those supplies Liscus. <LH>,train,anticipation
...,...,...,...,...
1455558,0x321566,I'm SO HAPPY!!! #NoWonder the name of this sho...,train,joy
1455559,0x38959e,In every circumtance I'd like to be thankful t...,train,joy
1455560,0x2cbca6,there's currently two girls walking around the...,train,joy
1455561,0x24faed,"Ah, corporate life, where you can date <LH> us...",train,joy


In [ ]:
emotion_csv.loc[emotion_csv['tweet_id']=='0x321566']

,tweet_id,emotion
697841,0x321566,joy


In [ ]:
# train_df = train_df.sample(frac=1)
# test_df = test_df.sample(frac=1)

print("Shape of Training df: ", train_df.shape)
print("Shape of Testing df: ", test_df.shape)

train_df.to_pickle("train_df.pkl") 
test_df.to_pickle("test_df.pkl")

Shape of Training df:  (1455563, 4)
Shape of Testing df:  (411972, 3)


In [122]:
## load a pickle file
train_df = pd.read_pickle("train_df.pkl")

train_df.drop('identification', axis=1, inplace=True)
train_df

,tweet_id,text,emotion
0,0x376b20,"People who post ""add me on #Snapchat"" must be ...",anticipation
1,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",sadness
2,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,fear
3,0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,joy
4,0x2c91a8,Still waiting on those supplies Liscus. <LH>,anticipation
...,...,...,...
1455558,0x321566,I'm SO HAPPY!!! #NoWonder the name of this sho...,joy
1455559,0x38959e,In every circumtance I'd like to be thankful t...,joy
1455560,0x2cbca6,there's currently two girls walking around the...,joy
1455561,0x24faed,"Ah, corporate life, where you can date <LH> us...",joy


In [123]:
test_df = pd.read_pickle("test_df.pkl")

test_df.drop('identification', axis=1, inplace=True)
test_df

,tweet_id,text
2,0x28b412,"Confident of your obedience, I write to you, k..."
4,0x2de201,"""Trust is not the same as faith. A friend is s..."
9,0x218443,When do you have enough ? When are you satisfi...
30,0x2939d5,"God woke you up, now chase the day #GodsPlan #..."
33,0x26289a,"In these tough times, who do YOU turn to as yo..."
...,...,...
1867525,0x2913b4,"""For this is the message that ye heard from th..."
1867529,0x2a980e,"""There is a lad here, which hath five barley l..."
1867530,0x316b80,When you buy the last 2 tickets remaining for ...
1867531,0x29d0cb,I swear all this hard work gone pay off one da...


In [124]:
# train_df.to_csv("train_df.csv", index=False, sep='\t')
# test_df.to_csv("test_df.csv", index=False, sep='\t')

##### ----- 文本向量化

大致步驟

1. 使用CountVectorizer()取得Bag of Word的model，並設定詞彙量'30000'，代表從dicitionary中取出頻率最高的前30000個詞彙

2. 用BOW.fit()來從'train_df['text']'中(學習)依照BOW的分詞方式將文本分詞並按照頻率排序，保留前30000個頻率最高的詞彙

3. 接著使用BOW.transform(train_df['text'])將每個文本轉換成向量(前30000個頻率最高的詞彙)，構成document-term matrix

In [125]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer

# nltk.download('punkt')

## build analyzers (bag-of-words)
## Only use the Top 30000 frequency? token in dictionary.
## Tokenizer
BOW = CountVectorizer(max_features=30000, tokenizer=nltk.word_tokenize) 

## apply analyzer to training data
## Learning Top 500 token frequency? in specific fit dictionary.
BOW.fit(train_df['text'])

# train_data_BOW_features = BOW.transform(train_df['text'])

## check dimension
# train_data_BOW_features.shape

c:\Users\Martin\Desktop\Master_1\Virtualenv\Python311_ML\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CountVectorizer(max_features=30000,
                tokenizer=<function word_tokenize at 0x0000023434683CE0>)

In [126]:
# print(pd.DataFrame(train_data_BOW_features.toarray(), columns = BOW.get_feature_names_out()))

# # Show feature names for features [100:110]
# feature_names = BOW.get_feature_names_out()
# print("Feature names for features [100:110]:", feature_names[100:110])

In [127]:
# ---training data (features)
X_train = BOW.transform(train_df['text'])
# ---training labels
y_train = train_df['emotion']

##### ----- 使用多層感知器(Multilayer Perceptron, MLP)

詳細內容請看程式碼內註解

會使用MLP是因為我認為大型文本分類對於傳統演算法的分類器來說過於複雜，

難以去擬和大部分的情況，因此我這邊就採用NN(Neural Network)的方式來學習並預測。

那會採用regressor而不是classifier是因為實際跑一遍之後發現classifier的效果比較差，

細部原因可能與model內部的結構有關，很難單單以最後一層是回歸還是分類來來說明對結果的影響。

In [128]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
import keras

label_encoder = LabelEncoder()
label_encoder.fit(y_train)

def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

# 因為MLP的輸入必須是向量，因此需要對label做one-hot encoding
y_train = label_encode(label_encoder, y_train)

In [129]:
from sklearn.neural_network import MLPRegressor

# Create an MLP regression model with verbose training
MLP_regressor = MLPRegressor(activation='logistic',
                             solver='adam',
                             alpha=0.00007,
                             max_iter=30,
                             hidden_layer_sizes=(32,32,32),
                             verbose=True,
                             early_stopping=False)

# Training the MLP regression model
MLP_regressor.fit(X_train, y_train)

Iteration 1, loss = 0.04110118
Iteration 2, loss = 0.03890811
Iteration 3, loss = 0.03873735
Iteration 4, loss = 0.03847380
Iteration 5, loss = 0.03821872
Iteration 6, loss = 0.03804256
Iteration 7, loss = 0.03794462
Iteration 8, loss = 0.03788994
Iteration 9, loss = 0.03787218
Iteration 10, loss = 0.03788978
Iteration 11, loss = 0.03793331
Iteration 12, loss = 0.03799559
Iteration 13, loss = 0.03807854


c:\Users\Martin\Desktop\Master_1\Virtualenv\Python311_ML\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (13) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(activation='logistic', alpha=7e-05,
             hidden_layer_sizes=(32, 32, 32), max_iter=13, verbose=True)

In [130]:
# Predictions on training and testing data
y_train_pred_MLP = MLP_regressor.predict(X_train)
print(y_train_pred_MLP[0:2])

[[0.03036042 0.04806706 0.44980247 0.02728538 0.16222733 0.09594797
  0.11556524 0.06905279]
 [0.00719277 0.11988142 0.17258166 0.03894605 0.1495606  0.36844629
  0.05513797 0.08797384]]


In [131]:
pred_train_result = label_decode(label_encoder, y_train_pred_MLP)
pred_train_result[:2]

array(['disgust', 'sadness'], dtype=object)

In [132]:
# ---training data (features)
# 使用deocde將預測的向量轉換回實際的label(emotion)
X_test = BOW.transform(test_df['text'])
y_test_pred_MLP = MLP_regressor.predict(X_test)
pred_test_result = label_decode(label_encoder, y_test_pred_MLP)
pred_test_result[:2]

array(['anticipation', 'anticipation'], dtype=object)

In [133]:
submission_df = pd.DataFrame({
    'id':test_df['tweet_id'],
    'emotion':pred_test_result,
})

submission_df

,id,emotion
2,0x28b412,anticipation
4,0x2de201,anticipation
9,0x218443,joy
30,0x2939d5,joy
33,0x26289a,trust
...,...,...
1867525,0x2913b4,anticipation
1867529,0x2a980e,anticipation
1867530,0x316b80,sadness
1867531,0x29d0cb,joy


In [134]:
submission_df.to_csv('submission.csv', index=False)